# Improvement Trial With Longer Reward Delay

I will increase the reward for delayed score. If this doesn't work, I will restart the trials, but use multiple lagged scores instead of recording win, because final score is what is being maximized anyway.

In [1]:
import gpu_rec
from constants import *
from copy import deepcopy
from random import shuffle
from game import Game
import numpy as np
import sys

Using TensorFlow backend.


In [2]:
#game is now using lag=1 instead of lag=2 for q-score
#maybe this will help guide it to higher point values in short term, 
#with less opportunities for probabilities straying
test_config = deepcopy(DEFAULT_GAME_CONFIG)
test_config['memory'] = 1
test_config['discount'] = 0.1

init_game = Game(n_players=5, q_lag=4, config=test_config)
#goal: eventually train players to not want to grab more tickets

players = init_game.players
#former will be shuffled
player_list = [p for p in players]
overall_player_wins = []
overall_player_scores = []
overall_winning_scores = []
overall_game_turns = []

for i, player in enumerate(player_list):
    player.ai.load_models('ai_h5/p%d_test09_%%s.h5' % i)



In [3]:
def which_min(x):
    min_val = min(x)
    return x.index(min_val)

def which_max(x):
    max_val = max(x)
    return x.index(max_val)

def aid_weakest_ai(players, wins):
    players[which_min(wins)].ai.receive_models_from_other_ai(players[which_max(wins)].ai)
    

In [4]:
temperatures = [ 0.1, 0.05, 0.01, 0.01, 0.001, 0.00001, 0.00001, 0., 0., 0.]
n_trials_per_cycle = 140
n_cycles = 3

for temp_index, temperature in enumerate(temperatures):
    print '---TEMPERATURE: %.03f---' % temperature
    print '+IDX=%d+' % temp_index
    if temp_index > 2:
        test_config['discount'] = 0.4
    test_config['temperature'] = temperature
    player_wins = [0 for _ in players]
    winning_scores = []
    player_scores = [[] for _ in players]
    game_turns = []
    for cycle in range(n_cycles):
        print 'in cycle %d' % cycle
        for i in range(n_trials_per_cycle):
            sys.stdout.write('.')
            shuffle(players)
            game = Game(pre_existing_players=players, config=test_config, separated_models=True)
            game.run(False)
            for i, player in enumerate(player_list):
                player_wins[i]+= player.win
                player_scores[i].append(player.total_points)
            winning_scores.append(game.winning_score)
            game_turns.append(game.turn)
        #don't train player 0, train player 1 partially, and train players 2 and 3 much more
        for i, player in enumerate(player_list):
            if i==1:
                player.ai.train_win()
                player.ai.train_q()
            elif i > 1:
                player.ai.train_win(20)
                player.ai.train_q(20)
    for player in player_list:
        player.ai.reset_history()
    aid_weakest_ai(player_list[1:], player_wins[1:])
    overall_player_wins.append(player_wins)
    overall_player_scores.append(player_scores)
    overall_winning_scores.append(winning_scores)
    overall_game_turns.append(game_turns)
    #print out some summaries
    print 'WINNING SCORE AVERAGE: %.1f' % np.mean(winning_scores)
    print 'NUMBER OF TURNS AVERAGE: %.1f' % np.mean(game_turns)
    for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % (np.mean(player_wins[i])/(float(n_trials_per_cycle*n_cycles)))
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])
        

---TEMPERATURE: 0.100---
+IDX=0+
in cycle 0
............................................................................................................................................in cycle 1
............................................................................................................................................in cycle 2
............................................................................................................................................WINNING SCORE AVERAGE: 41.3
NUMBER OF TURNS AVERAGE: 234.3
---PLAYER 0 STATS---
WIN AVERAGE: 0.000
SCORE AVERAGE: -33.5
---PLAYER 1 STATS---
WIN AVERAGE: 0.257
SCORE AVERAGE: 23.0
---PLAYER 2 STATS---
WIN AVERAGE: 0.281
SCORE AVERAGE: 22.3
---PLAYER 3 STATS---
WIN AVERAGE: 0.264
SCORE AVERAGE: 21.1
---PLAYER 4 STATS---
WIN AVERAGE: 0.226
SCORE AVERAGE: 20.5
---TEMPERATURE: 0.050---
+IDX=1+
in cycle 0
..............................................................................................................

ai.py:414: RuntimeWarning: invalid value encountered in divide
  return (selected)/np.sum(selected)


....................................................................................................

ai.py:386: RuntimeWarning: invalid value encountered in less
  return np.random.choice(len(new_probs), p=new_probs)


.......................................in cycle 1
............................................................................................................................................in cycle 2
............................................................................................................................................WINNING SCORE AVERAGE: 44.1
NUMBER OF TURNS AVERAGE: 226.5
---PLAYER 0 STATS---
WIN AVERAGE: 0.000
SCORE AVERAGE: -25.2
---PLAYER 1 STATS---
WIN AVERAGE: 0.255
SCORE AVERAGE: 26.6
---PLAYER 2 STATS---
WIN AVERAGE: 0.257
SCORE AVERAGE: 25.7
---PLAYER 3 STATS---
WIN AVERAGE: 0.264
SCORE AVERAGE: 26.9
---PLAYER 4 STATS---
WIN AVERAGE: 0.255
SCORE AVERAGE: 26.7
---TEMPERATURE: 0.010---
+IDX=3+
in cycle 0
............................................................................................................................................in cycle 1
........................................................................................................

ai.py:413: RuntimeWarning: overflow encountered in power
  selected = x**(1./(temperature)**0.5)


..........................................................................................................................

ValueError: probabilities do not sum to 1

going to patch class method to avoid issues with probs not adding to 1; this will be updated in the main file, but I don't want to restart the kernel

In [12]:
import new

EPSILON = 1e-7

def min_normalize(x, epsilon=EPSILON):
	return (x - np.min(x))/(epsilon + np.max(x) - np.min(x))

def normalize(x, temperature):
	if temperature < EPSILON:
		selected = x + EPSILON >=np.max(x)*1 
		return (selected+EPSILON)/np.sum(selected+EPSILON)
	else:
		x = x + EPSILON 
		selected = x**(1./(temperature)**0.5)
		return (selected)/np.sum(selected)

def PATCH_decide_based_on_p_and_q(self, probabilities, q_scores, player):
		"""
		uses temperature and discount parameters
		"""
		temperature = player.game.config['temperature']
		discount = player.game.config['discount']
		#print probabilities 
		#print q_scores 
		new_probs = normalize(probabilities * discount + (min_normalize(q_scores)),
			temperature)
		#print new_probs
		try:
			return np.random.choice(len(new_probs), p=new_probs)
		except ValueError:
			return np.random.choice(len(new_probs))

In [14]:
for player in player_list:
    player.ai.decide_based_on_p_and_q = new.instancemethod(PATCH_decide_based_on_p_and_q, player.ai, None)

In [15]:
temperatures = [ 0., 0., 0.]
n_trials_per_cycle = 140
n_cycles = 3

for temp_index, temperature in enumerate(temperatures):
    print '---TEMPERATURE: %.03f---' % temperature
    print '+IDX=%d+' % temp_index
    #if temp_index > 2:
    #    test_config['discount'] = 0.4
    test_config['temperature'] = temperature
    player_wins = [0 for _ in players]
    winning_scores = []
    player_scores = [[] for _ in players]
    game_turns = []
    for cycle in range(n_cycles):
        print 'in cycle %d' % cycle
        for i in range(n_trials_per_cycle):
            sys.stdout.write('.')
            shuffle(players)
            game = Game(pre_existing_players=players, config=test_config, separated_models=True)
            game.run(False)
            for i, player in enumerate(player_list):
                player_wins[i]+= player.win
                player_scores[i].append(player.total_points)
            winning_scores.append(game.winning_score)
            game_turns.append(game.turn)
        #don't train player 0, train player 1 partially, and train players 2 and 3 much more
        for i, player in enumerate(player_list):
            if i==1:
                player.ai.train_win()
                player.ai.train_q()
            elif i > 1:
                player.ai.train_win(20)
                player.ai.train_q(20)
    for player in player_list:
        player.ai.reset_history()
    aid_weakest_ai(player_list[1:], player_wins[1:])
    overall_player_wins.append(player_wins)
    overall_player_scores.append(player_scores)
    overall_winning_scores.append(winning_scores)
    overall_game_turns.append(game_turns)
    #print out some summaries
    print 'WINNING SCORE AVERAGE: %.1f' % np.mean(winning_scores)
    print 'NUMBER OF TURNS AVERAGE: %.1f' % np.mean(game_turns)
    for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % (np.mean(player_wins[i])/(float(n_trials_per_cycle*n_cycles)))
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])
        

---TEMPERATURE: 0.000---
+IDX=0+
in cycle 0
............................................................................................................................................in cycle 1
............................................................................................................................................in cycle 2
............................................................................................................................................WINNING SCORE AVERAGE: 55.4
NUMBER OF TURNS AVERAGE: 217.2
---PLAYER 0 STATS---
WIN AVERAGE: 0.000
SCORE AVERAGE: -17.7
---PLAYER 1 STATS---
WIN AVERAGE: 0.252
SCORE AVERAGE: 34.5
---PLAYER 2 STATS---
WIN AVERAGE: 0.260
SCORE AVERAGE: 33.4
---PLAYER 3 STATS---
WIN AVERAGE: 0.250
SCORE AVERAGE: 35.1
---PLAYER 4 STATS---
WIN AVERAGE: 0.286
SCORE AVERAGE: 35.1
---TEMPERATURE: 0.000---
+IDX=1+
in cycle 0
..............................................................................................................

In [17]:
print game.turn
print i
print cycle
print temp_index
for player in player_list:
    print player
    print len(player.determine_possible_tracks())

215
4
2
2
player with 0/7 completed/uncompleted tickets, 44 train cards, 21 cars, and 
		-32 total points
66
player with 0/2 completed/uncompleted tickets, 6 train cards, 1 cars, and 
		48 total points
5
player with 0/6 completed/uncompleted tickets, 28 train cards, 9 cars, and 
		5 total points
73
player with 0/2 completed/uncompleted tickets, 8 train cards, 1 cars, and 
		33 total points
5
player with 0/2 completed/uncompleted tickets, 16 train cards, 15 cars, and 
		51 total points
76


In [18]:
np.mean(overall_winning_scores,1)

array([ 41.27619048,  41.05      ,  44.05238095,  43.63095238,
        51.76190476,  55.35      ,  56.53095238,  56.05      ])

In [19]:
np.mean(overall_player_scores, ( 2,))

array([[-33.50952381,  23.03333333,  22.33333333,  21.05      ,  20.5452381 ],
       [-31.26904762,  23.30238095,  25.32857143,  24.68095238,
         23.84047619],
       [-25.17619048,  26.60714286,  25.7       ,  26.90238095,
         26.66904762],
       [-25.72380952,  26.19761905,  26.2547619 ,  26.82857143,
         28.02380952],
       [-19.64761905,  31.96904762,  32.96904762,  31.70952381,
         31.71904762],
       [-17.7047619 ,  34.53333333,  33.41428571,  35.10238095,
         35.12619048],
       [-16.93333333,  36.45238095,  37.41428571,  35.11190476,
         33.99047619],
       [-16.5452381 ,  34.46190476,  35.89047619,  35.21904762,
         34.63571429]])

In [20]:
for i, player in enumerate(player_list):
    player.ai.save_models('ai_h5/p%d_test11_%%s.h5' % i)

Right now the issue is that the AI sees no advantage to building longer tracks. I propose altering the win "discount" to a longer, but feasible lag, as well as adding extra lags for smaller track sizes so that the network can learn to build smaller paths first. This will increase computation overhead linearly, but memory overhead will only have a very modest increase.